In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

import h5py

Using TensorFlow backend.


In [2]:
data = h5py.File("../dataset/data_sample.hdf5", "r")
print("data keys: "+str(list(data.keys())))

data keys: ['images', 'labels', 'observation_days']


In [3]:
images = data["images"][:]
print("image shape: (sample, x_size, y_size, epoch) = "+str(images.shape))

labels = data["labels"][:]
obs_days = data["observation_days"][:]

image shape: (sample, x_size, y_size, epoch) = (72000, 21, 21, 48)


In [5]:
labels.shape

(72000,)

In [ ]:
flatten_data = images.transpose(0, 3, 1, 2).reshape((72000*48, 21, 21))
flatten_data.shape